# Testing the Fluspect Leaf Reflectance model

In [1]:
# Add PATH (adapt!)
push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/Leaf/");
push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/Utils/");

In [5]:
pwd()

"/Volumes/cfranken/code/gitHub/LSM-SPAM/src/test"

In [6]:
# Use FluspectMod
using FluspectMod

┌ Info: Precompiling FluspectMod [top-level]
└ @ Base loading.jl:1186


In [16]:
# Define state vector
x = [2.,100.,10.,0.000,0.1,0.01,0.01,0.5]
"""
Definition below (needed to add as vector)
N = x[1]
Cab = x[2]
Car = x[3]
Ant = x[4]
Brown = x[5]
Cw = x[6]
Cm = x[7]
Cx = x[8]
Kcaro = (1.0-Cx).* KcaV + Cx .* KcaZ;
"""

"Definition below (needed to add as vector)\nN = x[1]\nCab = x[2]\nCar = x[3]\nAnt = x[4]\nBrown = x[5]\nCw = x[6]\nCm = x[7]\nCx = x[8]\nKcaro = (1.0-Cx).* KcaV + Cx .* KcaZ;\n"

In [17]:
# Run Fluspect:
RT,Mf,Mb = FluspectMod.fluspect(x)

0.01


([0.0433423 9.21214e-7; 0.0432896 7.97374e-7; … ; 0.100329 0.101107; 0.0995877 0.100245], [2.64069e-9 2.6568e-9 … 3.51955e-10 3.42111e-10; 3.10563e-9 3.12385e-9 … 4.80304e-10 4.6721e-10; … ; 1.06572e-8 1.08062e-8 … 1.66849e-8 1.65806e-8; 0.0 0.0 … 0.0 0.0], [4.82508e-7 4.91412e-7 … 2.36546e-9 2.04044e-9; 6.04963e-7 6.16145e-7 … 3.26164e-9 2.81323e-9; … ; 1.83427e-8 1.86204e-8 … 2.18608e-8 2.14164e-8; 0.0 0.0 … 0.0 0.0])

In [18]:
using Plots
plotly()

Plots.PlotlyBackend()

In [19]:
plot(FluspectMod.lambda, RT[:,1], xlabel="Wavelength", ylabel="Reflectance/1-T", label="Reflectance")
plot!(FluspectMod.lambda, 1 .-RT[:,2], xlabel="Wavelength", ylabel="Reflectance/1-T", label="1-Transmission")

In [23]:
lam = FluspectMod.lambda;
Iwle   = findall((lam.>=FluspectMod.minwle) .& (lam.<=FluspectMod.maxwle));
Iwlf   = findall((lam.>=FluspectMod.minwlf) .& (lam.<=FluspectMod.maxwlf));

In [33]:
in = 50
plot(lam[Iwlf], Mb[:,in], xlabel="Wavelength", label="SIF upward response to light at "*string(lam[Iwle[in]]))
in = 250
plot!(lam[Iwlf], Mb[:,in], xlabel="Wavelength", label="SIF upward response to light at "*string(lam[Iwle[in]]))

in = 50
plot!(lam[Iwlf], Mf[:,in], xlabel="Wavelength", label="SIF downward response to light at "*string(lam[Iwle[in]]))
in = 250
plot!(lam[Iwlf], Mf[:,in], xlabel="Wavelength", label="SIF downward response to light at "*string(lam[Iwle[in]]))
